In [ ]:
# ===============================
# 1. SETUP & IMPORTS
# ===============================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# ===============================
# 2. LOAD SAMPLE DATASET
# We'll use CIFAR-10 (can simulate recyclable vs non-recyclable classes)
# ===============================
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Select only 2 classes for simplicity: 1 (automobile) = non-recyclable, 2 (bird) = recyclable
train_filter = np.where((y_train == 1) | (y_train == 2))[0]
test_filter = np.where((y_test == 1) | (y_test == 2))[0]

x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

# Normalize pixel values
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels: 1 → 0 (non-recyclable), 2 → 1 (recyclable)
y_train = np.where(y_train == 2, 1, 0)
y_test = np.where(y_test == 2, 1, 0)

# ===============================
# 3. BUILD A LIGHTWEIGHT MODEL
# ===============================
model = keras.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===============================
# 4. TRAIN MODEL
# ===============================
history = model.fit(x_train, y_train, epochs=5, batch_size=64,
                    validation_data=(x_test, y_test))

# Evaluate model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\nTest Accuracy: {test_acc*100:.2f}%")

# ===============================
# 5. CONVERT MODEL TO TENSORFLOW LITE
# ===============================
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save TFLite model
with open("edge_ai_recycle_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Model converted to TensorFlow Lite successfully!")

# ===============================
# 6. TESTING THE TFLITE MODEL
# ===============================
interpreter = tf.lite.Interpreter(model_path="edge_ai_recycle_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Pick a random test image
test_image = np.expand_dims(x_test[5], axis=0).astype(np.float32)

# Run inference
interpreter.set_tensor(input_details[0]['index'], test_image)
interpreter.invoke()
prediction = interpreter.get_tensor(output_details[0]['index'])[0][0]

label = "Recyclable" if prediction > 0.5 else "Non-Recyclable"

plt.imshow(x_test[5])
plt.title(f"Prediction: {label}")
plt.axis("off")
plt.show()
